In [1]:
# Prequisites

# %pip install --upgrade azure-ai-ml
# %pip install --upgrade azure-identity
# %pip install --upgrade datasets==2.9.0
# %pip install py7zr

In [13]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    workspace = workspace_ml_client.workspace_name
    resource_group = workspace_ml_client.resource_group_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, workspace
    )

# replace with the registry name
nemo_registry = "nvidia-ai" #"azureml"

# registry where models are present
model_registry = "azureml-preview-test1"

nemo_registry_ml_client = MLClient(
    credential, subscription_id, resource_group, registry_name=nemo_registry
)
model_registry_ml_client = MLClient(
    credential, subscription_id, resource_group, registry_name=model_registry
)
nemo_registry_ml_client

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fd1c959c100>,
         subscription_id=d4d34678-c0d7-4d69-a257-366e3cb4a7d8,
         resource_group_name=registry-builtin-nvidia-eastus,
         workspace_name=None)

## Creating Computes

In [14]:
# If you already have a gpu cluster, mention it here. Else will create a new one with the name 'ghyadav-westus-a100'
# model will only run on an a100 instance

compute_cluster = "nemo-westus"
try:
    compute = workspace_ml_client.compute.get(compute_cluster)
    print(f"GPU compute '{compute_cluster}' found.")
except Exception as ex:
    print(f"GPU compute '{compute_cluster}' not found. Creating new one.")
    compute = AmlCompute(
        name=compute_cluster,
        size="STANDARD_ND96AMSR_A100_V4",
        max_instances=2,  # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

# generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))


# This is the number of GPUs in a single node of the selected 'vm_size' compute.
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpus_per_node = 1  # default value
gpu_count_found = False
ws_computes = workspace_ml_client.compute.list_sizes()
for ws_compute in ws_computes:
    if ws_compute.name.lower() == compute.size.lower():
        gpus_per_node = ws_compute.gpus
        print(f"Number of GPUs in compute {ws_compute.name} are {ws_compute.gpus}")
# if gpu_count_found not found, then print an error
if gpus_per_node > 0:
    gpu_count_found = True
else:
    gpu_count_found = False
    print(f"No GPUs found in compute. Number of GPUs in compute {compute.size} 0.")

GPU compute 'nemo-westus' found.
Number of GPUs in compute Standard_NC96ads_A100_v4 are 4


## Input Data for Evaluation

In [15]:
# download the dataset using the helper script. This needs datasets library: https://pypi.org/project/datasets/
import os
from datasets import load_dataset, get_dataset_split_names
dataset_dir = "pubmed-dataset"
dataset_name = "pubmed_qa"
# create the download directory if it does not exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)


# import hugging face datasets library

split = 'train' # Only test available
dataset = load_dataset(dataset_name, 'pqa_labeled', split=split)
# save the split of the dataset to the download directory as json lines file
dataset.to_json(os.path.join(dataset_dir, f"{split}.jsonl"))
# print dataset features


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 27.73ba/s]


2201997

In [5]:
dataset

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})

In [6]:
# evaluation_dataset = 'text-generation-data.jsonl' # evaluation_dataset_path
# prompt_data = '' # prompt_data_path
# load the ./samsum-dataset/train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd
evaluation_dataset = "./pubmed-dataset/test/test.jsonl"
pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
df = pd.read_json("./pubmed-dataset/train.jsonl", lines=True)
df = df[:10]

In [7]:
def form_question(obj):
    st = ""
    st += f"QUESTION: {obj['question']}\n"
    st += "CONTEXT: "
    for i in range(len(obj['context']['labels'])):
        st += f"{obj['context']['contexts'][i]}\n"
    st += f"TARGET: the answer to the question given the context is (yes|no|maybe): "
    return st

result = []
for i, row in df.iterrows():
    result.append(form_question(row))
data = pd.DataFrame({'input': result})
data

,input
0,"QUESTION: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?\nCONTEXT: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.\nThe following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos and examined. Mitochondrial dynamics were delineated into four categories (M1-M4) based on characteristics including distribution, motility, and membrane potential (ΔΨm). A TUNEL assay showed fragmented nDNA in a gradient over these mitochondrial stages. Chloroplasts and transvacuolar strands were also examined using live cell imaging. The possible importance of mitochondrial permeability transition pore (PTP) formation during PCD was indirectly examined via in vivo cyclosporine A (CsA) treatment. This treatment resulted in lace plant leaves with a significantly lower number of perforations compared to controls, and that displayed mitochondrial dynamics similar to that of non-PCD cells.\nTARGET: the answer to the question given the context is (yes|no|maybe):"
1,"QUESTION: Landolt C and snellen e acuity: differences in strabismus amblyopia?\nCONTEXT: Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differences in the lower visual acuity range cannot be excluded. In this study, visual acuity measured with the Snellen E was compared to the Landolt C acuity.\n100 patients (age 8 - 90 years, median 60.5 years) with various eye disorders, among them 39 with amblyopia due to strabismus, and 13 healthy volunteers were tested. Charts with the Snellen E and the Landolt C (Precision Vision) which mimic the ETDRS charts were used to assess visual acuity. Three out of 5 optotypes per line had to be correctly identified, while wrong answers were monitored. In the group of patients, the eyes with the lower visual acuity, and the right eyes of the healthy subjects, were evaluated.\nDifferences between Landolt C acuity (LR) and Snellen E acuity (SE) were small. The mean decimal values for LR and SE were 0.25 and 0.29 in the entire group and 0.14 and 0.16 for the eyes with strabismus amblyopia. The mean difference between LR and SE was 0.55 lines in the entire group and 0.55 lines for the eyes with strabismus amblyopia, with higher values of SE in both groups. The results of the other groups were similar with only small differences between LR and SE.\nTARGET: the answer to the question given the context is (yes|no|maybe):"
2,"QUESTION: Syncope during bathing in infants, a pediatric form of water-induced urticaria?\nCONTEXT: Apparent life-threatening events in infants are a difficult and frequent problem in pediatric practice. The prognosis is uncertain because of risk of sudden infant death syndrome.\nEight infants aged 2 to 15 months were admitted during a period of 6 years; they suffered from similar maladies in the bath: on immersion, they became pale, hypotonic, still and unreactive; recovery took a few s

In [8]:
frac = 1
evaluation_dataset = "./pubmedqa-dataset/test/test_frac.jsonl"
os.makedirs("./pubmedqa-dataset/test", exist_ok=True)
data.sample(frac=frac).to_json(
    evaluation_dataset, orient="records", lines=True
)

## Loading model from Registry

In [16]:
model_name = "Nemotron-3-8B-Base-4k"
model_version = "latest"

nemo_model_object = nemo_registry_ml_client.models.get(model_name, label="latest")

# Prompt

In [17]:
prompt_path = "prompts/prompt-qna.txt"


## Submitting Evaluation Pipeline

In [20]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# fetch the pipeline component
pipeline_component_func = nemo_registry_ml_client.components.get(
    name="nemo_qna_evaluation", label="latest"
    # name = "nemo_prediction_base", label="latest"
)
openai_params='{"type":"azure_open_ai","model_name":"gpt-35-turbo","deployment_name":"gpt-35-turbo","questions":"input","contexts":"input"}'



# define the pipeline job
@pipeline()
def evaluation_pipeline(mlflow_model):
    evaluation_job = pipeline_component_func(
        dataset_path=Input(type=AssetTypes.URI_FOLDER, path="./pubmedqa-dataset/test/"),
        model_path=Input(type=AssetTypes.TRITON_MODEL, path=f"{nemo_model_object.id}"),
        openai_config_params=openai_params,
        # prompt_schema=Input(type=AssetTypes.URI_FILE, path=prompt_path), # Not required for text-gen task
        # prediction_column_name="output",   
    )
    return {"evaluation_result": evaluation_job.outputs.evaluation_result}

In [22]:
experiment_name = "nemo-qna-eval-pipeline"
pipeline_jobs = []

pipeline_object = evaluation_pipeline()

# don't reuse cached results from previous jobs
pipeline_object.settings.force_rerun = True
pipeline_object.settings.default_compute = compute_cluster
pipeline_job = workspace_ml_client.jobs.create_or_update(
    pipeline_object, experiment_name=experiment_name
)
# add model['name'] and pipeline_job.name as key value pairs to a dictionary
pipeline_jobs.append({"model_name": model_name, "job_name": pipeline_job.name})
# wait for the pipeline job to complete
workspace_ml_client.jobs.stream(pipeline_job.name)

RunId: helpful_salt_g3w0n26ltf
Web View: https://ml.azure.com/runs/helpful_salt_g3w0n26ltf?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/nvidia/workspaces/nvidia-eus

Streaming logs/azureml/executionlogs.txt

[2023-11-15 12:05:17Z] Submitting 1 runs, first five are: 6306a868:f7ebc092-9058-4590-afce-04d70408657e
